## Dependencias

In [3]:
%pip install pandas nltk matplotlib firebase-admin python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Pipeline

### 1. Preprocesamiento
1. Bajar todas las votaciones desde Firebase y dejarla en una lista
2. Construir un `pandas.DataFrame` con los datos de Firebase
3. Agregar una dimensión al data frame con vector de _stemmed tokens_
4. Quitar subconjunto de palabras a ignorar ('señor', 'inciso', etc.)

### 2. Análisis Exploratorio
1. Filtrar por años de interés
2. Visualizar distribuciones de atributos de interés
3. Hacer primeras preguntas a poder responder con los datos

### Conexión Firebase

In [2]:
from dotenv import dotenv_values, find_dotenv

import firebase_admin # type: ignore
from firebase_admin import credentials, firestore

async def firebase_connection():
    config = dotenv_values(find_dotenv())

    firebase_credentials_file_path = config["FIREBASE_CREDENTIALS"]
    cred = credentials.Certificate(firebase_credentials_file_path)

    if not firebase_admin._apps:
        firebase_admin.initialize_app(cred)

    db = firestore.client()

    return db

db = await firebase_connection()
db

### Traer Votaciones de Firebase

In [4]:
import pandas as pd

votaciones_docs = db.collection("votaciones").stream()
votaciones = list(map(lambda doc: doc.to_dict(), votaciones_docs))

votaciones_df = pd.DataFrame(votaciones)

In [13]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer

nltk.download("stopwords")

stemmer = SnowballStemmer("spanish")
stop_words = set(stopwords.words("spanish"))
additional_stop_words = set([
  'ley',
  'señor',
  'proyect',
  'boletin',
  'aprob',
  'tramit',
  'establec',
  'nº',
  'constitucional',
  'segund',
  'indic',
  'comision',
  'modif',
  'articul',
  'republ',
  'inform',
  'acuerd',
  'general',
  'votacion',
  'president',
  'part',
  'senador',
  'prim',
  'nacional',
  'honor',
  'solicit',
  'particul',
  'public'
])

def filter_useful_tokens(token):
  return len(token) > 1 and token not in stop_words and token not in additional_stop_words and not token.isdigit()

def tokenize(materia):
  numero_boletin = RegexpTokenizer(r"[Bb]olet[íi]n.*\d+").tokenize(materia)

  if len(numero_boletin) > 0:
    numero_boletin = "".join(re.findall(r"[0-9-]+", numero_boletin[0]))
  else:
    numero_boletin = ""

  tokenized_materia = RegexpTokenizer(r"\w+").tokenize(materia)
  tokenized_materia = WordPunctTokenizer().tokenize(" ".join(tokenized_materia))
  tokenized_materia = [token.lower() for token in tokenized_materia]
  tokenized_materia.append(numero_boletin)
  tokenized_materia = list(filter(filter_useful_tokens, tokenized_materia))

  return tokenized_materia

def stem(tokenized_materia):
  stemmed_materia = [
      stemmer.stem(token) for token in tokenized_materia if token.lower() not in stop_words
  ]
  stemmed_materia = list(filter(filter_useful_tokens, stemmed_materia))
  return stemmed_materia

votaciones_df['tokenized_materia'] = votaciones_df['materia'].apply(tokenize)
votaciones_df['stemmed_materia'] = votaciones_df['tokenized_materia'].apply(stem)
votaciones_df

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oxfist/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,fecha_str,resultado,boletin_numero,id,legislatura_id,url,sesion_numero,tramite_constitucional,detalle,sesion_id,...,articulo,camara,tipo,votos_no,quorum,votos_si,materia,votos_abs,tokenized_materia,stemmed_materia
0,12/03/2013 19:55,,6190-19,5000,484,https://www.senado.cl/appsenado/index.php?mo=s...,1,,"{'NO': ['Alvear V., Soledad', 'Cantero O., Car...",6799,...,,SENADO,,22,Cuatro séptimos Q.C.,5,"Votaciòn de la indicación número 200 bis.-, re...",1,"[votaciòn, indicación, número, bis, renovada]","[votaciòn, numer, bis, renov]"
1,13/03/2013 16:33,,7818-14,5001,484,https://www.senado.cl/appsenado/index.php?mo=s...,2,,"{'NO': [], 'SI': ['Allende B., Isabel', 'Bianc...",6801,...,,SENADO,,0,Cuatro séptimos Q.C.,25,Votación que aprueba en particular el proyecto...,0,"[votación, aprueba, particular, proyecto, renu...","[aprueb, renuev, proced, regulariz, ampliacion..."
2,13/03/2013 17:11,,6967-06,5002,484,https://www.senado.cl/appsenado/index.php?mo=s...,2,,"{'NO': [], 'SI': ['Allende B., Isabel', 'Alvea...",6801,...,,SENADO,,0,Mayoría simple,29,Votación que aprueba en general y en particula...,0,"[votación, aprueba, particularel, proyecto, se...","[aprueb, particularel, decl, feri, dia, juni, ..."
3,19/03/2013 13:07,,6190-19,5003,484,https://www.senado.cl/appsenado/index.php?mo=s...,3,,"{'NO': ['Allende B., Isabel', 'Gómez U., José ...",6803,...,,SENADO,,8,Cuatro séptimos Q.C.,26,Aprobación del inciso primero del artículo 15 ...,0,"[aprobación, inciso, primero, artículo, conten...","[incis, primer, conten, pas, ser, 1º, permit, ..."
4,19/03/2013 13:11,,6190-19,5004,484,https://www.senado.cl/appsenado/index.php?mo=s...,3,,"{'NO': [], 'SI': ['Alvear V., Soledad', 'Bianc...",6803,...,,SENADO,,0,Cuatro séptimos Q.C.,27,Aprobación del incisos segundo y tercero del a...,1,"[aprobación, incisos, segundo, tercero, artícu...","[incis, tercer, conten, pas, ser, 1º, permit, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4579,05/03/2024 20:21,,,9661,503,https://www.senado.cl/appsenado/index.php?mo=s...,101,,"{'NO': ['Coloma C., Juan Antonio', 'Ebensperge...",9625,...,,SENADO,,3,Q.C.,36,"Enmiendas por mayorías, entre ellas, normas de...",1,"[enmiendas, mayorías, normas, quórum, aprobadas]","[enmiend, mayor, norm, quorum]"
4580,06/03/2024 15:38,,2.524-12,9662,503,https://www.senado.cl/appsenado/index.php?mo=s...,103,,"{'NO': [], 'SI': ['Allende B., Isabel', 'Arave...",9627,...,,SENADO,,0,Mayoría simple,24,\n Proyecto de acuerdo de los Honorables Sena...,0,"[proyecto, acuerdo, honorables, senadores, señ...","[nuñez, araven, carvajal, gatic, pascual, prov..."
4581,06/03/2024 16:23,,16576-08,9663,503,https://www.senado.cl/appsenado/index.php?mo=s...,103,,"{'NO': [], 'SI': ['Aravena A., Carmen Gloria',...",9627,...,,SENADO,,0,Mayoría simple,32,"Proyecto de ley, iniciado en Mensaje de Su Exc...",1,"[proyecto, iniciado, mensaje, excelencia, pres...","[inici, mensaj, excelent, divers, cuerp, legal..."
4582,06/03/2024 17:18,,16408-05,9664,503,https://www.senado.cl/appsenado/index.php?mo=s...,103,,"{'NO': [], 'SI': ['Allende B., Isabel', 'Arave...",9627,...,,SENADO,,0,Q.C.,37,"Proyecto de ley, en primer trámite constitucio...",0,"[proyecto, primer, trámite, modifica, diversos...","[divers, cuerp, legal, objet, adopt, med, comb..."


In [24]:
votaciones_df['fecha'] = pd.to_datetime(votaciones_df['fecha_str'], format='%d/%m/%Y %H:%M').dt.date

In [25]:
start_date = pd.to_datetime('11/03/2018 00:00:00', format='%d/%m/%Y %H:%M:%S').date()
end_date = pd.to_datetime('10/03/2024 23:59:59', format='%d/%m/%Y %H:%M:%S').date()

votaciones_df = votaciones_df[(votaciones_df['fecha'] >= start_date) & (votaciones_df['fecha'] <= end_date)]
votaciones_df

,fecha_str,resultado,boletin_numero,id,legislatura_id,url,sesion_numero,tramite_constitucional,detalle,sesion_id,...,articulo,camara,tipo,votos_no,quorum,votos_si,materia,votos_abs,tokenized_materia,fecha
2066,13/03/2018 16:49,,11465-22,7134,489,https://www.senado.cl/appsenado/index.php?mo=s...,1,,"{'NO': [], 'SI': ['Allamand Z., Andrés', 'Alle...",7979,...,,SENADO,,0,Mayoría simple,35,Perfecciona los beneficios otorgados a bombero...,0,"[perfeccion, benefici, otorg, bomber, accident...",2018-03-13
2067,14/03/2018 16:22,,10217-15,7135,489,https://www.senado.cl/appsenado/index.php?mo=s...,2,,"{'NO': ['Allamand Z., Andrés', 'Aravena A., Ca...",7981,...,,SENADO,,14,Mayoría simple,10,Informe de la Comisión Mixta constituida para ...,0,"[mixt, constitu, resolv, divergent, suscit, tr...",2018-03-14
2068,14/03/2018 16:52,,11269-05,7136,489,https://www.senado.cl/appsenado/index.php?mo=s...,2,,"{'NO': [], 'SI': ['Allamand Z., Andrés', 'Alle...",7981,...,,SENADO,,0,Cuatro séptimos Q.C.,37,"Proyecto de ley, en segundo trámite constituci...",0,"[moderniz, legisl, bancari]",2018-03-14
2069,14/03/2018 17:12,,10456-15,7137,489,https://www.senado.cl/appsenado/index.php?mo=s...,2,,"{'NO': [], 'SI': ['Allamand Z., Andrés', 'Alle...",7981,...,,SENADO,,0,Cuatro séptimos Q.C.,37,"Proyecto de ley, iniciado en Moción del Honora...",0,"[inici, mocion, navarr, pen, radiodifusion, au...",2018-03-14
2070,14/03/2018 17:19,,10456-15,7138,489,https://www.senado.cl/appsenado/index.php?mo=s...,2,,"{'NO': ['Guillier Á., Alejandro', 'Quintana L....",7981,...,,SENADO,,2,Mayoría simple,26,"Proyecto de ley, en primer trámite constitucio...",1,"[pen, radiodifusion, autoriz, von, baer, mont,...",2018-03-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4579,05/03/2024 20:21,,,9661,503,https://www.senado.cl/appsenado/index.php?mo=s...,101,,"{'NO': ['Coloma C., Juan Antonio', 'Ebensperge...",9625,...,,SENADO,,3,Q.C.,36,"Enmiendas por mayorías, entre ellas, normas de...",1,"[enmiend, mayor, norm, quorum]",2024-03-05
4580,06/03/2024 15:38,,2.524-12,9662,503,https://www.senado.cl/appsenado/index.php?mo=s...,103,,"{'NO': [], 'SI': ['Allende B., Isabel', 'Arave...",9627,...,,SENADO,,0,Mayoría simple,24,\n Proyecto de acuerdo de los Honorables Sena...,0,"[nuñez, araven, carvajal, gatic, pascual, prov...",2024-03-06
4581,06/03/2024 16:23,,16576-08,9663,503,https://www.senado.cl/appsenado/index.php?mo=s...,103,,"{'NO': [], 'SI': ['Aravena A., Carmen Gloria',...",9627,...,,SENADO,,0,Mayoría simple,32,"Proyecto de ley, iniciado en Mensaje de Su Exc...",1,"[inici, mensaj, excelent, divers, cuerp, legal...",2024-03-06
4582,06/03/2024 17:18,,16408-05,9664,503,https://www.senado.cl/appsenado/index.php?mo=s...,103,,"{'NO': [], 'SI': ['Allende B., Isabel', 'Arave...",9627,...,,SENADO,,0,Q.C.,37,"Proyecto de ley, en primer trámite constitucio...",0,"[divers, cuerp, legal, objet, adopt, med, comb...",2024-03-06


In [26]:
from collections import Counter

counter = Counter(token for tokens in votaciones_df['tokenized_materia'] for token in tokens)

most_common_materias = counter.most_common(20)
most_common_materias

[('constitu', 291),
 ('rechaz', 263),
 ('discusion', 246),
 ('bien', 227),
 ('refund', 220),
 ('present', 211),
 ('si', 200),
 ('orden', 196),
 ('castr', 195),
 ('enmiend', 181),
 ('numer', 180),
 ('servici', 177),
 ('rincon', 172),
 ('nuev', 172),
 ('objet', 165),
 ('cre', 162),
 ('chil', 160),
 ('reform', 160),
 ('chahuan', 159),
 ('respect', 155)]